## Semantic Search - PATENTS coding according to ISCO

In [ ]:
%%capture
%pip install -U git+https://github.com/istat-methodology/semantic-search.git

This is needed on Azure

In [ ]:
%pip install openpyxl

In [ ]:
import re
import pandas as pd

from semantic_search.data import build_corpus
from semantic_search.local import LocalKnowledgeBase

In [ ]:
# Azure specific paths
BASE_DIR = "/home/azureuser/cloudfiles/code/Users/mbruno/ai-patents-exposure/"

# Local environment
# BASE_DIR = ""


In [ ]:
ISCO_PATH = BASE_DIR + "resources/classification/ISCO-08_structure_and_definitions.xlsx"
PATENTS_SAMPLE_PATH = BASE_DIR + "sample/patents_sample.xlsx"
OUTPUT_PATH_SEMANTIC = BASE_DIR + "output/patents_classified_semantic.csv"

In [ ]:
TOP_K = 5  # Number of top results to consider for each query

### ISCO & PATENTS pre-processing 

Read and pre-process ISCO

In [ ]:
# Read Excel
isco_df = pd.read_excel(ISCO_PATH)

# Drop columns we won't use (silently ignore if missing)
DROP_COLS = ["Tasks include", "Included occupations", "Excluded occupations", "Notes"]
isco_df.drop(columns=DROP_COLS, inplace=True, errors='ignore')

# Keep only Level >= 3 (minor + unit groups) and make a defensive copy
sub_major_df = isco_df.loc[isco_df["Level"] >= 3].copy()

# Normalize code column to string and trim
sub_major_df["ISCO 08 Code"] = sub_major_df["ISCO 08 Code"].astype(str).str.strip()

# Derive 3-digit ISCO code:
#  - remove all non-digits (handles formats like '221.1', '221-10', etc.)
#  - take the first 3 digits (we'll drop rows that don't yield 3 digits)
sub_major_df["isco3"] = (
    sub_major_df["ISCO 08 Code"]
      .str.replace(r"\D", "", regex=True)
      .str[:3]
)

# Simple text cleaner: collapse multiple spaces, strip; empty string for NaN
def _clean_text(s: object) -> str:
    if pd.isna(s):
        return ""
    s = str(s).replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Clean fields needed downstream
sub_major_df["Title EN"]   = sub_major_df["Title EN"].map(_clean_text)
sub_major_df["Definition"] = sub_major_df["Definition"].map(_clean_text)

# Keep only well-formed 3-digit codes (safety)
sub_major_df = sub_major_df[sub_major_df["isco3"].str.len() == 3].copy()


Build ISCO clean dataset (needed by semantic_search to build the knowledge base)

In [ ]:
# One title per 3-digit code from Level == 3; drop empty titles
titles = (
    sub_major_df.loc[sub_major_df["Level"] == 3, ["isco3", "Title EN"]]
      .drop_duplicates("isco3")
)
titles = titles[titles["Title EN"].str.strip().ne("")].set_index("isco3")

# Aggregate all definitions (from Level >= 3) under the same 3-digit code
definitions = (
    sub_major_df
      .groupby("isco3", as_index=True)["Definition"]
      .apply(lambda x: _clean_text(" ".join([t for t in x if t])))
      .to_frame()
)

# Join definitions and titles; INNER keeps only codes with a (non-empty) title
isco_clean_df = definitions.join(titles, how="inner").reset_index()

# Build the descriptor string used by semantic search
#    (markdown ** around the title is intentional for visual emphasis)
isco_clean_df["descriptor"] = (
    "**" + isco_clean_df["Title EN"].str.strip() + "**. " +
    isco_clean_df["Definition"].str.strip()
).str.replace(r"\s+\.", ".", regex=True).str.strip()

# Tidy column names and order for downstream use
isco_clean_df = (
    isco_clean_df
      .rename(columns={"isco3": "ISCO3", "Title EN": "title", "Definition": "definition"})
      .loc[:, ["ISCO3", "title", "definition", "descriptor"]]
      .sort_values("ISCO3")
      .reset_index(drop=True)
)

In [ ]:
isco_clean_df.head()

Read sample patents and generate descriptor (needed by semantic search)

In [ ]:
patents_df = pd.read_excel(PATENTS_SAMPLE_PATH)
# Drop unnecessary columns
if "description" in patents_df.columns:
    patents_df = patents_df.drop("description", axis=1)

# Descriptor column is used in semantic search
patents_df["descriptor"] =  "**"+ patents_df["title"].str.strip() + "**. " + patents_df["abstract"].str.strip()


In [ ]:
patents_df.head()

### Semantic Search

1. Semantic Search - Build Corpus

In [ ]:
# Texts for embedding/search
texts = isco_clean_df["descriptor"].tolist()

# Sequential numeric IDs (1..N) — sorted by ISCO3 already in previous step
ids = list(range(len(isco_clean_df)))

# Metadata: include both 3-digit ISCO code and title
metadata=[{"code": c, "title": t} for c, t in zip(isco_clean_df["ISCO3"], isco_clean_df["title"])]

In [ ]:
# Build the corpus
corpus = build_corpus(
    texts=texts,
    ids=ids,
    metadata=metadata
)

2. Semantic Search - Build Knowledge Base (runs only on GPU)

In [ ]:
base = LocalKnowledgeBase(
    corpus=corpus,
    model_id="BAAI/bge-m3"
)

3. Extract the list of QUERIES -> The descriptor field in the PATENTS dataframe

In [ ]:
queries = patents_df["descriptor"].tolist()

4. Now, we can extract the results for each query. We will first extract the top n results, and then extract the unique codes.

In [ ]:
results = base.search(queries, top_k=TOP_K)

We need to define a function to parse the results and extract the unique codes.

In [ ]:
def parse_results(results):
    outs = []
    for result in results:

        max_scores = {}
        for r in result:
            code = r.metadata["code"]
            score = r.score

            if code not in max_scores or score > max_scores[code]:
                max_scores[code] = score

        outs.append(max_scores)

    return outs

In [ ]:
def expand_scores(parsed_results, top_k=TOP_K, code_prefix="code_", score_prefix="score_"):
    """
    Convert a list of dicts (code -> score) into a wide DataFrame with
    columns code_1, score_1, ..., code_k, score_k (sorted by descending score).
    Missing entries are filled with NaN.

    Parameters
    ----------
    parsed_results : list[dict]
        Each element corresponds to one patent's result: {code: score, ...}
    top_k : int | None
        Number of top entries to keep. If None, keeps all entries (wide as max length).
    code_prefix, score_prefix : str
        Column name prefixes.

    Returns
    -------
    pd.DataFrame
        One row per patent, with columns code_i / score_i.
    """
    rows = []
    max_len = 0

    for r in parsed_results:
        # be forgiving if r is None or not a dict
        items = list(getattr(r, "items", lambda: [])())
        # sort by score desc
        items.sort(key=lambda x: x[1], reverse=True)

        if top_k is not None:
            items = items[:top_k]

        row = {}
        for i, (code, score) in enumerate(items, start=1):
            row[f"{code_prefix}{i}"]  = str(code) if code is not None else None
            row[f"{score_prefix}{i}"] = float(score) if score is not None else None
        rows.append(row)
        max_len = max(max_len, len(items))

    df = pd.DataFrame(rows)

    # Ensure consistent column order
    k = top_k if top_k is not None else max_len
    ordered_cols = [c for i in range(1, k+1) for c in (f"{code_prefix}{i}", f"{score_prefix}{i}")]
    df = df.reindex(columns=ordered_cols)

    return df

In [ ]:
parsed_results = parse_results(results)

In [ ]:
scores_wide = expand_scores(parsed_results, top_k=TOP_K)

# append to your patents dataframe
patents_with_scores = pd.concat([patents_df.reset_index(drop=True), scores_wide], axis=1)

# optional: round score columns for readability (doesn't affect numeric type)
score_cols = [c for c in patents_with_scores.columns if c.startswith("score_")]
patents_with_scores[score_cols] = patents_with_scores[score_cols].round(4)


In [ ]:
patents_with_scores.to_csv(OUTPUT_PATH_SEMANTIC, index=False)